Characteristics for question 1:
income, population, density, average housing price

In [3]:
import pandas as pd
import numpy as np

Reading all the Uber data from 2014

In [2]:
uber_apr14 = pd.read_csv('data/uber/uber-raw-data-apr14.csv')
uber_may14 = pd.read_csv('data/uber/uber-raw-data-may14.csv')
uber_jun14 = pd.read_csv('data/uber/uber-raw-data-jun14.csv')
uber_jul14 = pd.read_csv('data/uber/uber-raw-data-jul14.csv')
uber_aug14 = pd.read_csv('data/uber/uber-raw-data-aug14.csv')
uber_sep14 = pd.read_csv('data/uber/uber-raw-data-sep14.csv')

In [3]:
uber_apr14.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


Concatanate all the dataframes together into one dataframe containing all Uber data from 2014

In [4]:
uber_14 = pd.concat([uber_apr14, uber_may14, uber_jun14, uber_jul14, uber_aug14, uber_sep14])

In [5]:
len(uber_14)

4534327

In [6]:
uber_14.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


Write Latitute and Longitude for rides to a json and excel file (Used for Tableau HeatMap)

In [8]:
uber_loc = uber_14[["Lat", "Lon"]]
uber_loc.to_json("uber_loc.json",orient='values')

writer = pd.ExcelWriter("uber_loc.xlsx")
uber_loc.to_excel(writer, index=False)
writer.save()

Read aggregate data (total number of rides) for Uber in 2015 and lookup information on neighborboods

In [4]:
uber_janjun15 = pd.read_csv('../data/uber/uber-raw-data-janjune-15.csv')
lookup = pd.read_csv('../data/taxi-zone-lookup.csv')
uber_janjun15.head()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID
0,B02617,2015-05-17 09:47:00,B02617,141
1,B02617,2015-05-17 09:47:00,B02617,65
2,B02617,2015-05-17 09:47:00,B02617,100
3,B02617,2015-05-17 09:47:00,B02774,80
4,B02617,2015-05-17 09:47:00,B02617,90


In [5]:
lookup.head()

,LocationID,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights


Renamed locationID column to maintain consistency. Merged aggregate data and lookup information on locationID

In [6]:
lookup.rename(columns={'LocationID': 'locationID'}, inplace=True)
uber_janjun15 = uber_janjun15.merge(lookup,on='locationID')

In [7]:
uber_janjun15.head()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID,Borough,Zone
0,B02617,2015-05-17 09:47:00,B02617,141,Manhattan,Lenox Hill West
1,B02617,2015-05-17 09:50:00,B02617,141,Manhattan,Lenox Hill West
2,B02617,2015-05-17 09:53:00,B02617,141,Manhattan,Lenox Hill West
3,B02617,2015-05-17 10:00:00,B02617,141,Manhattan,Lenox Hill West
4,B02617,2015-05-17 10:00:00,B02617,141,Manhattan,Lenox Hill West


Filter dataframe to only contain data points on rides given in NYC

In [8]:
uber_janjun15 = uber_janjun15[['Pickup_date','Borough','Zone']]
uber_janjun15 = uber_janjun15[uber_janjun15['Borough'] == 'Manhattan']

Counting the number of rides given in each neighborhood

In [9]:
uber_agg = uber_janjun15['Zone'].value_counts()
uber_agg = uber_agg.to_frame()
uber_agg.rename(columns={'Zone': 'Number of Rides'}, inplace=True)
uber_agg[:5] # Top five neighborhoods

,Number of Rides
Midtown Center,460732
TriBeCa/Civic Center,420356
Union Sq,419045
East Village,407591
West Village,323989


Reseting index and renaming previous index column

In [10]:
uber_agg = uber_agg.reset_index()
uber_agg.rename(columns={'index': 'Zone'}, inplace=True)
uber_agg.head()

,Zone,Number of Rides
0,Midtown Center,460732
1,TriBeCa/Civic Center,420356
2,Union Sq,419045
3,East Village,407591
4,West Village,323989


Adding information on population density, median household income, median age for males and females for each of the top 5 neighborhoods

In [17]:
top_5 = uber_agg[0:5]
top_5.is_copy = False
top_5.loc[:,'Population Density'] = pd.Series([47793,52346,41579,84277,75501],index=top_5.index)
top_5.loc[:,'Median Household Income'] = pd.Series([122586,222849,153694,76016,124638],index=top_5.index)
top_5.loc[:,'Median Age (Males)'] = pd.Series([40,36.5,36.3,35.1,39.9],index=top_5.index)
top_5.loc[:,'Median Age (Females)'] = pd.Series([38.6,36.3,34.9,36.5,37.3],index=top_5.index)
top_5

,Zone,Number of Rides,Population Density,Median Household Income,Median Age (Males),Median Age (Females)
0,Midtown Center,460732,47793,122586,40.0,38.6
1,TriBeCa/Civic Center,420356,52346,222849,36.5,36.3
2,Union Sq,419045,41579,153694,36.3,34.9
3,East Village,407591,84277,76016,35.1,36.5
4,West Village,323989,75501,124638,39.9,37.3


Saving dataframes to excel sheets

In [64]:
top_5_pop = top_5[['Zone','Population Density']]
top_5_pop.to_excel('top_5_population.xlsx',index=False)

top_5_inc = top_5[['Zone','Median Household Income']]
top_5_inc.to_excel('top_5_income.xlsx',index=False)

top_5_male = top_5[['Zone','Median Age (Males)']]
top_5_male.to_excel('top_5_male.xlsx',index=False)

top_5_female = top_5[['Zone','Median Age (Females)']]
top_5_female.to_excel('top_5_female.xlsx',index=False)

In [59]:
low_5 = uber_agg[len(uber_agg)-11:]
low_5['Population Density'] = [113670,0,118751,49823,0,0,45186,0,49823,0,0]
low_5['Median Household Income'] = [40857,0,40979,50909,0,0,129352,0,50909,0,0]
low_5['Median Age (Males)'] = [32.9,0,32.6,35.2,0,0,31.2,0,35.2,0,0]
low_5['Median Age (Females)'] = [40.5,0,38.2,38.7,0,0,31.4,0,38.7,0,0]
low_5 = low_5[low_5['Population Density'] > 0]
low_5

/home/vinitha910/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/vinitha910/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/vinitha910/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,Zone,Number of Rides,Population Density,Median Household Income,Median Age (Males),Median Age (Females)
56,Hamilton Heights,34290,113670,40857,32.9,40.5
58,Manhattanville,17802,118751,40979,32.6,38.2
59,Inwood,15138,49823,50909,35.2,38.7
62,Battery Park,2579,45186,129352,31.2,31.4
64,Inwood Hill Park,1200,49823,50909,35.2,38.7


In [65]:
low_5_pop = low_5[['Zone','Population Density']]
low_5_pop.to_excel('low_5_population.xlsx',index=False)

low_5_inc = low_5[['Zone','Median Household Income']]
low_5_inc.to_excel('low_5_income.xlsx',index=False)

low_5_male = low_5[['Zone','Median Age (Males)']]
low_5_male.to_excel('low_5_male.xlsx',index=False)

low_5_female = low_5[['Zone','Median Age (Females)']]
low_5_female.to_excel('low_5_female.xlsx',index=False)